# Implementing Convolutional Neural Network

#### Installing essential libraries

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report, confusion_matrix

#### Reading the dataset

In [ ]:
df=pd.read_csv("C:/Users/riddh/OneDrive/Documents/Projects/Churn/Preprocessed_data.xls")

#### Splitting the data

In [ ]:
X = df.drop('Churn', axis=1)
y = df['Churn']

#### Split data into training, validation, and test sets

In [ ]:
# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#### Check the dimension of the sets

In [ ]:
print('X_train:',np.shape(X_train))
print('y_train:',np.shape(y_train))
print('X_test:',np.shape(X_test))
print('y_test:',np.shape(y_test))

#### Checking Distribution of Classes

In [ ]:
import numpy as np
unique, counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

#### Reshaping of Data

In [ ]:
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = np.array(X_val).reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Check the dimensions of the sets
print('X_train:', np.shape(X_train))
print('y_train:', np.shape(y_train))
print('X_val:', np.shape(X_val))
print('y_val:', np.shape(y_val))
print('X_test:', np.shape(X_test))
print('y_test:', np.shape(y_test))

#### This code converts the X_train data by first cleaning it to replace any boolean values with integers, then converting the cleaned data into a NumPy array of type float32, and finally converting that array into a TensorFlow tensor.

In [ ]:
X_train_cleaned = np.array([[int(x) if isinstance(x, bool) else x for x in row] for row in X_train])
X_train_cleaned = np.array(X_train_cleaned, dtype=np.float32)
X_train = tf.convert_to_tensor(X_train_cleaned)

In [ ]:
X_val_cleaned = np.array([[int(x) if isinstance(x, bool) else x for x in row] for row in X_val])
X_val_cleaned = np.array(X_val_cleaned, dtype=np.float32)
X_val = tf.convert_to_tensor(X_val_cleaned)

In [ ]:
X_test_cleaned = np.array([[int(x) if isinstance(x, bool) else x for x in row] for row in X_test])
X_test_cleaned = np.array(X_test_cleaned, dtype=np.float32)
X_test = tf.convert_to_tensor(X_test_cleaned)

#### Model Designing

In [ ]:
# Build CNN model
model = Sequential([
    Conv1D(filters=128, kernel_size=3, activation='sigmoid', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='sigmoid'),
    MaxPooling1D(pool_size=2),
    
    Flatten(),
    Dense(128, activation='sigmoid'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

#### Compiling the model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Print a summary of the model architecture including the types of layers,
#output shapes, and the number of parameters in each layer
model.summary()

In [ ]:
# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Fit Model
epochs = 100
history = model.fit(X_train, y_train, validation_split=0.2, epochs=epochs, batch_size=32, 
                   callbacks=[early_stopping])

#### Evaluate the Model

In [ ]:
# Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

In [ ]:
yhat_test_prob = model.predict(X_test)
yhat_test = (yhat_test_prob > 0.5).astype(int)

from sklearn.metrics import confusion_matrix, classification_report
conf_matrix = confusion_matrix(y_test, yhat_test)
print('Confusion Matrix:')
print(conf_matrix)

print('Classification Report:')
print(classification_report(y_test, yhat_test))


In [ ]:
from sklearn.metrics import accuracy_score
# Printing the accuracy score
print('Accuracy:')
print(float(accuracy_score(y_test, yhat_test))*100,'%')

In [ ]:
# Printing the keys of the history object returned during model training
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt

# Define the epoch range based on the length of the loss history
epoch_range = range(1, len(history.history['loss']) + 1)

# Plot the training and validation loss over epochs
plt.plot(epoch_range, history.history['loss'], label='Train')   # Training loss
plt.plot(epoch_range, history.history['val_loss'], label='Validation')  # Validation loss

# Adding labels and title to the plot
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid()
plt.xlim((1, len(history.history['loss'])))  # Ensure x-axis limit matches the number of epochs
plt.legend()
plt.title('Loss vs Epochs')

# Displaying the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Define the epoch range based on the length of the accuracy history
epoch_range = range(1, len(history.history['accuracy']) + 1)

# Plot the training and validation accuracy over epochs
plt.plot(epoch_range, history.history['accuracy'], label='Train')   # Training accuracy
plt.plot(epoch_range, history.history['val_accuracy'], label='Validation')  # Validation accuracy

# Adding labels and title to the plot
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid()
plt.xlim((1, len(history.history['accuracy'])))  # Ensure x-axis limit matches the number of epochs
plt.legend()
plt.title('Accuracy vs Epochs')

# Displaying the plot
plt.show()